<a href="https://colab.research.google.com/github/lorenzopaoria/Smoking-detection-and-distance-analysis/blob/main/model_train_cigarette.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Train a model for sigarette detection

In [2]:
import torch
import torchvision
import psutil
import os
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from torch.utils.data import DataLoader, Dataset
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision import transforms
from PIL import Image
import numpy as np
import time
import matplotlib.pyplot as plt
from tqdm import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
class CigaretteDataset(Dataset):
    def __init__(self, coco_annotation_file, image_dir, transform=None):
        self.coco = COCO(coco_annotation_file)
        self.image_dir = image_dir
        self.transform = transform if transform is not None else transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        cat_ids = self.coco.getCatIds(catNms=['cigarette'])
        if not cat_ids:
            raise ValueError("No 'cigarette' category found in COCO file.")
        self.image_ids = list(set(self.coco.getImgIds(catIds=cat_ids)))

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        if idx >= len(self):
            raise IndexError(f"Index {idx} out of range for dataset of size {len(self)}")

        img_id = self.image_ids[idx]
        img_info = self.coco.loadImgs(img_id)[0]

        try:
            image = Image.open(f"{self.image_dir}/{img_info['file_name']}").convert("RGB")
        except Exception as e:
            print(f"Error loading image {img_info['file_name']}: {str(e)}")
            return None

        ann_ids = self.coco.getAnnIds(imgIds=img_id)
        annotations = self.coco.loadAnns(ann_ids)
        boxes, labels = [], []

        for ann in annotations:
            cat_name = self.coco.loadCats(ann['category_id'])[0]['name']
            if cat_name == 'cigarette':
                x, y, w, h = ann['bbox']
                boxes.append([x, y, x + w, y + h])
                labels.append(1)  # 1 for cigarette, 0 is background

        if len(boxes) == 0:
            return None

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)

        try:
            image = self.transform(image)
        except Exception as e:
            print(f"Error in transform for image {img_info['file_name']}: {str(e)}")
            return None

        target = {
            'boxes': boxes,
            'labels': labels,
            'image_id': torch.tensor([img_id])
        }

        return image, target

In [5]:
def collate_fn(batch):
    return tuple(zip(*[b for b in batch if b is not None]))

In [6]:
def check_system_usage():
    print(f"CPU Usage: {psutil.cpu_percent()}%")
    print(f"RAM Usage: {psutil.virtual_memory().percent}%")
    if torch.cuda.is_available():
        print(f"GPU Memory Allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
        print(f"GPU Memory Reserved: {torch.cuda.memory_reserved() / 1e9:.2f} GB")

In [7]:
def evaluate_model(model, dataset, device):
    print("\n=== Starting Validation ===")
    model.eval()
    total_val_loss = 0
    valid_batches = 0
    coco_dt = []
    coco_gt = dataset.coco

    val_loader = DataLoader(
        dataset,
        batch_size=8,
        shuffle=False,
        collate_fn=collate_fn,
        num_workers=2,
        pin_memory=True
    )

    with torch.no_grad():
        for images, targets in tqdm(val_loader, desc="Validating"):
            if not images or not targets:
                continue

            images = [image.to(device) for image in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            try:
                loss_dict = model(images, targets)
                if isinstance(loss_dict, dict) and loss_dict:
                    losses = sum(loss for loss in loss_dict.values())
                    if not torch.isnan(losses) and not torch.isinf(losses):
                        total_val_loss += losses.item()
                        valid_batches += 1

                predictions = model(images)

                for idx, prediction in enumerate(predictions):
                    image_id = targets[idx]['image_id'].item()
                    boxes = prediction['boxes']
                    scores = prediction['scores']
                    labels = prediction['labels']

                    for box, score, label in zip(boxes, scores, labels):
                        if score > 0.5:  # threshold
                            x1, y1, x2, y2 = box.tolist()
                            coco_dt.append({
                                'image_id': image_id,
                                'category_id': label.item(),
                                'bbox': [x1, y1, x2-x1, y2-y1],
                                'score': score.item()
                            })

            except RuntimeError as e:
                if "out of memory" in str(e):
                    torch.cuda.empty_cache()
                    print("OOM error during validation, skipping batch...")
                    continue
                else:
                    raise e

    avg_val_loss = total_val_loss / valid_batches if valid_batches > 0 else float('inf')
    print(f"\nValidation Loss: {avg_val_loss:.4f}")

    # COCO evaluation
    if len(coco_dt) > 0:
        try:
            coco_dt = coco_gt.loadRes(coco_dt)
            coco_eval = COCOeval(coco_gt, coco_dt, iouType='bbox')
            coco_eval.evaluate()
            coco_eval.accumulate()
            coco_eval.summarize()
        except Exception as e:
            print(f"Error during COCO evaluation: {str(e)}")

    return avg_val_loss

In [8]:
def train_model(dataset, num_epochs=20, val_dataset=None, min_epochs=10, patience=6, improvement_threshold=0.1):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # 2 classes (background, cigarette)
    model = fasterrcnn_resnet50_fpn(weights='DEFAULT')
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes=2)
    model = model.to(device)

    data_loader = DataLoader(
        dataset,
        batch_size=8,
        shuffle=True,
        collate_fn=collate_fn,
        num_workers=2,
        pin_memory=True
    )

    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    scaler = torch.amp.GradScaler('cuda')

    best_val_loss = float('inf')
    epochs_no_improve = 0
    best_model_state = None
    previous_losses = []

    save_dir = '/content/drive/MyDrive/pth_cigarette_detect'
    os.makedirs(save_dir, exist_ok=True)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        valid_batches = 0

        for batch_idx, batch in enumerate(tqdm(data_loader, desc=f"Epoch {epoch+1}/{num_epochs}")):
            if batch is None or len(batch) != 2:
                continue

            images, targets = batch
            if not images or not targets:
                continue

            try:
                images = [image.to(device) for image in images]
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

                optimizer.zero_grad()

                if scaler is not None:
                    with torch.amp.autocast(device_type='cuda'):
                        loss_dict = model(images, targets)
                        losses = sum(loss for loss in loss_dict.values())
                    scaler.scale(losses).backward()
                    scaler.step(optimizer)
                    scaler.update()
                else:
                    loss_dict = model(images, targets)
                    losses = sum(loss for loss in loss_dict.values())
                    losses.backward()
                    optimizer.step()

                total_loss += losses.item()
                valid_batches += 1

            except RuntimeError as e:
                if "out of memory" in str(e):
                    if torch.cuda.is_available():
                        torch.cuda.empty_cache()
                    print(f"OOM error in batch {batch_idx}, skipping...")
                    continue
                else:
                    raise e

        if valid_batches > 0:
            avg_loss = total_loss / valid_batches
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

            # save checkpoint for each epoch
            epoch_path = os.path.join(save_dir, f'epoch_{epoch+1}.pth')
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': avg_loss
            }, epoch_path)
            print(f"Saved checkpoint to {epoch_path}")

            if val_dataset:
                val_loss = evaluate_model(model, val_dataset, device)
                previous_losses.append(val_loss)

                # check for best model
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    epochs_no_improve = 0
                    best_model_state = model.state_dict().copy()
                    best_model_path = os.path.join(save_dir, 'best_model.pth')
                    torch.save({
                        'epoch': epoch + 1,
                        'model_state_dict': best_model_state,
                        'optimizer_state_dict': optimizer.state_dict(),
                        'loss': avg_loss,
                        'val_loss': val_loss
                    }, best_model_path)
                    print(f"Saved new best model with validation loss: {val_loss:.4f}")
                else:
                    epochs_no_improve += 1

                # early stopping logic
                if epoch >= min_epochs and epochs_no_improve >= patience:
                    if len(previous_losses) >= patience:
                        min_recent_loss = min(previous_losses[-patience:])
                        max_recent_loss = max(previous_losses[-patience:])
                        if max_recent_loss - min_recent_loss < improvement_threshold:
                            print(f"\nEarly stopping triggered after epoch {epoch+1}")
                            print(f"No improvement of {improvement_threshold} or more in validation loss for {patience} epochs")
                            if best_model_state is not None:
                                model.load_state_dict(best_model_state)
                            break

    # save final model
    final_model_path = os.path.join(save_dir, "fasterrcnn_cigarette_final.pth")
    torch.save(model.state_dict(), final_model_path)
    print(f"Saved final model to {final_model_path}")
    return model

In [9]:
if __name__ == "__main__":
    train_image_dir = '/content/drive/MyDrive/Photo/train'
    train_coco_annotation_file = '/content/drive/MyDrive/Photo/train/_annotations.coco.json'

    valid_image_dir = '/content/drive/MyDrive/Photo/valid'
    valid_coco_annotation_file = '/content/drive/MyDrive/Photo/valid/_annotations.coco.json'

    dataset = CigaretteDataset(train_coco_annotation_file, train_image_dir)
    val_dataset = CigaretteDataset(valid_coco_annotation_file, valid_image_dir)

    model = train_model(
    dataset,
    num_epochs=20,           # Maximum number of epochs
    val_dataset=val_dataset,
    min_epochs=10,          # Minimum number of epochs before early stopping
    patience=6,             # Number of epochs to wait for improvement
    improvement_threshold=0.1 # Minimum improvement required
)

loading annotations into memory...
Done (t=3.13s)
creating index...
index created!
loading annotations into memory...
Done (t=0.94s)
creating index...
index created!
Using device: cuda


Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:01<00:00, 159MB/s]
Epoch 1/20: 100%|██████████| 92/92 [04:18<00:00,  2.81s/it]


Epoch 1/20, Loss: 0.3680
Saved checkpoint to /content/drive/MyDrive/pth_cigarette_detect/epoch_1.pth

=== Starting Validation ===


Validating: 100%|██████████| 18/18 [00:55<00:00,  3.11s/it]



Validation Loss: inf
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.11s).
Accumulating evaluation results...
DONE (t=0.04s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.101
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.257
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.046
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.266
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.411
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.062
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.136
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.136
 Average Recall     (AR) @[ IoU=0.50:0.95 | ar

...


Validation Loss: inf
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.11s).
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.161
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.303
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.153
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.454
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.553
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.088
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.183
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.183
 Average Recall     (AR) @[ IoU=0.50:0.95 | ar

Epoch 20/20: 100%|██████████| 92/92 [01:55<00:00,  1.25s/it]


Epoch 20/20, Loss: 0.0532
Saved checkpoint to /content/drive/MyDrive/pth_cigarette_detect/epoch_20.pth

=== Starting Validation ===


Validating: 100%|██████████| 18/18 [00:41<00:00,  2.29s/it]



Validation Loss: inf
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.11s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.169
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.308
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.173
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.477
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.584
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.092
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.191
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.191
 Average Recall     (AR) @[ IoU=0.50:0.95 | ar